In [0]:
%run "../includes/configuration"

In [0]:
v_source_file = dbutils.widgets.get("p_source_file")

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType,LongType,DateType

In [0]:
results_schama = StructType(fields=[StructField("resultId", IntegerType(), True),
                                    StructField("raceId", IntegerType(), False),
                                    StructField("driverId", IntegerType(), False),
                                    StructField("constructorId", IntegerType(), False),
                                    StructField("number", IntegerType(), False),
                                    StructField("grid", IntegerType(), False),
                                    StructField("position", IntegerType(), False),
                                    StructField("positionText", StringType(), False),
                                    StructField("positionOrder", IntegerType(), False),
                                    StructField("points", DoubleType(), False),
                                    StructField("laps", IntegerType(), False),
                                    StructField("time", StringType(), False),
                                    StructField("milliseconds", IntegerType(), False),
                                    StructField("fastestLap", IntegerType(), False),
                                    StructField("rank", IntegerType(), False),
                                    StructField("fastestLapTime", StringType(), False),
                                    StructField("fastestLapSpeed", StringType(), False),
                                    StructField("statusId", IntegerType(), False)])

In [0]:
results_df = spark.read \
    .option("header", "true") \
    .schema(results_schama) \
    .json(f"{raw_folder_path}/results.json")

# results_df = spark.read \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .json("/mnt/dldatabrickscoursedev001/raw/results.json")



In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

In [0]:
results_columnAlteration_df = results_df.withColumnRenamed("resultId", "result_id") \
                                        .withColumnRenamed("raceId", "race_id") \
                                        .withColumnRenamed("driverId", "driver_id") \
                                        .withColumnRenamed("constructorId", "constructor_id") \
                                        .withColumnRenamed("positionText", "position_text") \
                                        .withColumnRenamed("positionOrder", "position_order") \
                                        .withColumnRenamed("fastestLap", "fastest_lap") \
                                        .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                        .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                        .withColumn("data_source", lit(v_source_file)) \
                                        .drop("statusid")

In [0]:
%run "../functions/common_functions"

In [0]:
results_columnAlteration_df = add_ingest_date(results_columnAlteration_df)

In [0]:
results_columnAlteration_df.write \
    .mode("overwrite")  \
    .partitionBy("race_id") \
    .parquet(f"{processed_folder_path}/results")

In [0]:
spark.read.parquet(f"{processed_folder_path}/results/race_id=2/").display()

In [0]:
dbutils.notebook.exit("success")